# Student Loan Risk with Deep Learning

In [ ]:
# Imports
import pandas as pd
import tensorflow as tf
import sklearn as skl
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [ ]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

In [ ]:
# output_file = "student_loans.csv"
# loans_df.to_csv(output_file, index=False)

In [ ]:
# Review the data types associated with the columns
loans_df.dtypes

In [ ]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [ ]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
y[:5]

In [ ]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns="credit_ranking")

# Review the features DataFrame
X.head()

### Step 3: Split the features and target sets into training and testing datasets.


In [ ]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [ ]:
# Create a StandardScaler instance
# X_scaler = skl.preprocessing.StandardScaler()
X_scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler.fit(X_train)

# ~~Fit the scaler to the features training dataset~~
# Scale the features training and testing datasets
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [ ]:
# Define the number of inputs (features) to the model
input_nodes = len(X.columns)

# Review the number of features
input_nodes

In [ ]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes1 = 6

# Define the number of hidden nodes for the second hidden layer
hidden_nodes2 = 3

# Define the number of neurons in the output layer
output_node = 1

In [ ]:
# Create the Sequential model instance
nn_model = tf.keras.models.Sequential()

# Add the first hidden layer
# nn_model.add(tf.keras.layers.Dense(units=hidden_nodes1, input_dim=input_nodes, activation="relu"))
nn_model.add(Dense(units=hidden_nodes1, input_dim=input_nodes, activation="relu"))

# Add the second hidden layer
# nn_model.add(tf.keras.layers.Dense(units=hidden_nodes2, activation="relu"))
nn_model.add(Dense(units=hidden_nodes2, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
# nn_model.add(tf.keras.layers.Dense(units=output_node, activation="sigmoid"))
nn_model.add(Dense(units=output_node, activation="sigmoid"))

In [ ]:
# Display the Sequential model summary
nn_model.summary()

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [ ]:
# Compile the Sequential model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Fit the model using 50 epochs and the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [ ]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [ ]:
# Set the model's file path
file_path = Path("student_loans.keras")

# Export your model to a keras file
nn_model.save(file_path)


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [ ]:
# Set the model's file path
file_path = Path("student_loans.keras")

# Load the model to a new object
nn_model_imported = tf.keras.models.load_model(file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [ ]:
# Make predictions with the test data
predictions = nn_model_imported.predict(X_test_scaled, verbose=2)
# Display a sample of the predictions
predictions[:5]

In [ ]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame(predictions, columns=["predictions"])
predictions_df['predictions'] = round(predictions_df["predictions"],0)
predictions_df

### Step 4: Display a classification report with the y test data and predictions

In [ ]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_df["predictions"]))

---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**

To build a recommendation system for student loan options, we would need to collect specific data on the student and the different loan options available in order to personalize loan recommendations to each student's unique circumstances and characteristics. A student's age, gender, educational level, field of study, the institution they are attending, their current loan debt, and loan history would be relevant data to collect to match them with the appropriate lender, loam amount, interest rates, and repayment terms for the following reasons:
* Age: Younger and older students may be eligible for different options in relation to federal vs private loans. They also may have different financial needs, with older students qualifying for loans that require a more established credit history, and younger students benefiting from loans that do not require a credit check.
* Gender: Gender should not be used as a factor in determining loan eligibility. To enforce to this, data most be collected to provide insights into loan trends and ensure the system is not biased. Any disparities in loan approvals or termns can be identified and addressed.
* Educational Level: The level of education a student is pursuing can impact the amount of loan they are eligible for, the interest rates they qualify for, and the repayment terms available to them. For example, graduate students may be eligible for higher loan amounts than undergraduate students.
* Field of Study: The field of study a student is pursuing can impact their future earning potential, employment prospects, and ability to repay a loan. Students in high-demand fields may qualify for lower interest rates or more favorable repayment terms.
* Institution: The institution a student is attending can impact the cost of attendance, the availability of financial aid, and the types of loans available to them. For example, students attending a public university may have access to different loan options than students attending a private university.
* Current Loan Debt: The amount of current loan debt a student has can impact their eligibility for additional loans, the interest rates they qualify for, and the repayment terms available to them. Students with high levels of existing debt may be limited in the amount of additional loans they can take on.
* Loan History: A student's loan history, including their repayment behavior, credit score, and debt-to-income ratio, can impact their eligibility for future loans, the interest rates they qualify for, and the repayment terms available to them. Students with a history of on-time payments may qualify for lower interest rates, while students with a history of late payments may be limited in the loan options available to them.

**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**

A content-based filtering method suits the data better than collaborative filtering or context-based filtering by providing the following advantages:

* Personalization: features of the student such as credit score, income level, employment history, and past loan repayments provide personalized predictions for each student.
*  Attributes Focus: features like credit history, debt-to-income ratio, and employment stability are emphasized as the most indicative of repayment behavior.
*  Improved Accuracy: historical data of students with similar profiles and attributes can be used to enhance the accuracy of future behavior.
* Transparency: explanations based on attributes are easier to explain to stakeholders and students in comparison to the less transparent collaborative filtering or black-box machine learning models.
* Adaptibility: prediction updates based on the students improving or deteriorating situation ensure risk assessment remains current.
* Compliance: fair lending practices based on objective criteria related to student attributes are more likely to reduce bias and discrimination in loan recommendations.
* Scalability: quick and consistent content-based predictions even when dealing with large volumes of loan applications avoiding the need for collaborative-filtering method comparing every borrower against a large pool of other students

**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

 * Challenge 1: Data Sparsity due to incomplete financial history is challenge when building this kind of recommendation system. Student typically don't have long credit history, high income levels, or extensive financial data which are important features for predictive models. 
    * Solution: To mitigage the issue of data sparsity, alternative data sources such as academic performance, field of study, attendance records, and employment prospects in their field of study. Proxy variable or credit scoring models which rely on metrics more appropriate for students such as rent payment or utility bill payment history can serve as indicators of financial responsibility in the absence of traditional credit history.
* Challenge 2: Economic uncertainty and future employment risks can significantly impact a student's ability to repay a loan. A recommendation system that fails to account for fluctuations in the job market, changes in industry demand, and economic recessions might overestimate a student's ability to repay a loan, leading to increased default rates and financial distress for borrowers.
    * Solution: To address the challenge of economic uncertainty, the recommendation system should incorporate macroeconomic indicators, industry trends, and employment forecasts to predict future employment prospects and earning potential for students. By considering these external factors, the recommendation system can provide more accurate and reliable loan recommendations that align with a student's financial situation and future job prospects.